In [2]:
from xml.etree import ElementTree as ET
import pandas as pd
import folium
import json


def generate_coordinates_dictionary(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    tei_header = root.find('{http://www.tei-c.org/ns/1.0}teiHeader')
    if tei_header is not None:
        places = tei_header.findall('.//{http://www.tei-c.org/ns/1.0}place')
        coordinates_dict = {}

        for place in places:
            place_name_element = place.find(
                '{http://www.tei-c.org/ns/1.0}placeName')
            if place_name_element is not None:
                place_name = place_name_element.text
                place_type = place.get('type')
                geo_element = place.find(
                    '{http://www.tei-c.org/ns/1.0}location/{http://www.tei-c.org/ns/1.0}geo')
                if geo_element is not None:
                    coordinates = geo_element.text
                    if coordinates:
                        try:
                            lat, lon = coordinates.split(",")
                            # Convert to float and create a tuple
                            coordinates_dict[(place_name, place_type)] = (
                                float(lat), float(lon))
                            
                        except ValueError:
                            print(
                                f"Invalid coordinates format for {place_name}: {coordinates}")
                            
        m = folium.Map(location=[48.21095900638111,16.377296146797722], zoom_start=5)
        for key, value in coordinates_dict.items():
            if key[1] == 'city':
                folium.Circle(
                    radius=5000,
                    location=value,
                    popup=key[0],
                    color="#3186cc",
                    fill=True,
                    fill_color="#3186cc",
                    tooltip=key[0],
                ).add_to(m)
            else:
               folium.Marker(location=value, popup=key[0]).add_to(m)
        
        #modo per tracciare l'area dello stato
        try:
            with open('denmark-detailed-boundary_896.geojson', 'r', encoding='utf-8') as file:
                data = json.load(file)
                folium.GeoJson(data, name="denmark", tooltip="Denmark").add_to(m)
        except Exception as e:
            print(f"Error loading GeoJSON: {e}")

        return m.save("metascript_map.html")
        



generate_coordinates_dictionary("C:/Users/crosi/Documents/GitHub/metascript/Dream_Story.xml")

In [18]:
from xml.etree import ElementTree as ET
import pandas as pd
import folium
import json


def generate_coordinates_dictionary(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    tei_header = root.find('{http://www.tei-c.org/ns/1.0}teiHeader')
    if tei_header is not None:
        places = tei_header.findall('.//{http://www.tei-c.org/ns/1.0}place')
        coordinates_dict = {}

        for place in places:
            place_name_element = place.find(
                '{http://www.tei-c.org/ns/1.0}placeName')
            if place_name_element is not None:
                place_name = place_name_element.text
                place_type = place.get('type')
                geo_element = place.find(
                    '{http://www.tei-c.org/ns/1.0}location/{http://www.tei-c.org/ns/1.0}geo')
                if geo_element is not None:
                    coordinates = geo_element.text
                    if coordinates:
                        try:
                            lat, lon = coordinates.split(",")
                            # Convert to float and create a tuple
                            coordinates_dict[(place_name, place_type)] = (
                                float(lat), float(lon))
                            
                        except ValueError:
                            print(
                                f"Invalid coordinates format for {place_name}: {coordinates}")
                            
        m = folium.Map(location=[40.71258347482157, -74.0181331499278], zoom_start=6)
        for key, value in coordinates_dict.items():
            if key[1] == 'city':
                folium.Circle(
                    radius=5000,
                    location=value,
                    popup=key[0],
                    color="#3186cc",
                    fill=True,
                    fill_color="#3186cc",
                    tooltip=key[0],
                ).add_to(m)
            else:
               folium.Marker(location=value, popup=key[0]).add_to(m)
        
        #modo per tracciare l'area dello stato
        try:
                with open('connecticut.geojson', 'r', encoding='utf-8') as file:
                    data = json.load(file)
                    folium.GeoJson(data, name="connecticut", tooltip="Connecticut").add_to(m)
                with open('new jersey.geojson', 'r', encoding='utf-8') as file:
                    data = json.load(file)
                    folium.GeoJson(data, name="newjersey", tooltip="New Jersey").add_to(m)  
        except Exception as e:
            print(f"Error loading GeoJSON: {e}")

        return m
        
# m.save("metascript_map.html")


generate_coordinates_dictionary("C:/Users/crosi/Documents/GitHub/metascript/screenplay_1996.xml")